In [3]:
from pyspark.sql import DataFrame, SparkSession

spark = SparkSession.builder.appName('eda').getOrCreate()

In [4]:
import numpy as np
import pandas as pd

In [5]:
# make some dummy data

df = pd.DataFrame({
    'id': ['A'] * 100,
    'year': [2023] * 100,
    'month': np.linspace(1, 100, 100), # not real obviously
    'num1': np.random.normal(1000, 100, size=100),
    'num2': np.random.normal(10, 100, size=100),
    'cat1': np.random.choice(['a', 'b', 'c'], size=100),
    'cat2': np.random.choice(['A', 'B', 'C'], size=100),
    'bin_target': np.random.choice([0, 1],size=100, p=[0.2, 0.8]),
    'multi_target': np.random.choice([-1, 0, 1], size=100, replace=True, p=[0.2, 0.5, 0.3])
})

df.head()

,id,year,month,num1,num2,cat1,cat2,bin_target,multi_target
0,A,2023,1.0,1002.398970,78.289810,c,A,0,-1
1,A,2023,2.0,945.292032,-48.193934,a,A,1,1
2,A,2023,3.0,803.047617,-31.003851,a,A,0,1
3,A,2023,4.0,837.898480,-319.630657,a,A,1,1
4,A,2023,5.0,842.321657,-16.540610,c,C,1,-1


In [6]:
dd = spark.createDataFrame(df)

/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [7]:
dd.show()

+---+----+-----+------------------+--------------------+----+----+----------+------------+
| id|year|month|              num1|                num2|cat1|cat2|bin_target|multi_target|
+---+----+-----+------------------+--------------------+----+----+----------+------------+
|  A|2023|  1.0|1002.3989695645216|   78.28981021365396|   c|   A|         0|          -1|
|  A|2023|  2.0| 945.2920324240617|  -48.19393418442653|   a|   A|         1|           1|
|  A|2023|  3.0| 803.0476168393415|  -31.00385061597097|   a|   A|         0|           1|
|  A|2023|  4.0| 837.8984800348729| -319.63065707325995|   a|   A|         1|           1|
|  A|2023|  5.0| 842.3216571543076| -16.540609711770397|   c|   C|         1|          -1|
|  A|2023|  6.0| 843.5387529856456|  202.61459610296797|   b|   A|         1|           1|
|  A|2023|  7.0| 886.5776484723364|-0.09885695743611222|   c|   B|         1|           0|
|  A|2023|  8.0|1052.1454089179265|  -75.38275449654631|   c|   B|         1|           1|

# Box Plots

In [34]:
# need to do type chekcing
cols = ['num1',]# 'num2']

In [35]:
import plotly.graph_objects as go

cols_q = dd.approxQuantile(cols, [0.00, 0.01, 0.25, 0.50, 0.75, 0.99, 1.00], 0)
cols_q_dict = {}
for idx, col in enumerate(cols):
    cols_q_dict[col] = cols_q[idx]

# assume subplots are 3 wide, need to find num cols in subplot based on number columns in dataframe needed 
if len(cols) == 1:
    # no subplot needed

    q0, q01, q25, q50, q75, q99, q100 = cols_q[0]  
    fig = go.Figure(
        go.Box(
            q1=[q25],
            median=[q50],
            q3=[q75],
            upperfence=[q99],
            lowerfence=[q01]
        )
    )

In [36]:
from pyspark.sql import functions as F 

# need to do for individual columns
col = cols[0]
q_expr = F.expr(f'percentile_approx({col}, array(0.25, 0.50, 0.75))')
dd.groupBy('bin_target').agg(q_expr.alias('percentiles')).show()

+----------+--------------------+
|bin_target|         percentiles|
+----------+--------------------+
|         0|[935.376196164749...|
|         1|[953.810277191174...|
+----------+--------------------+



In [43]:
(dd.groupBy('bin_target')
    .agg(F.percentile_approx(col, 0.5))).show()

+----------+-----------------------------------+
|bin_target|percentile_approx(num1, 0.5, 10000)|
+----------+-----------------------------------+
|         0|                  991.0439847015035|
|         1|                 1009.3529238157965|
+----------+-----------------------------------+



In [104]:
# split by target

# by column
class Explorer:
    pass

# pyspark
def make_box(self, cols):
    quantile_dict = {}
    for col in cols:
        # q00, q01, q25, q50, q75, q99, q100 = 
        target_q = (dd.groupBy(self.target)
            .agg(
                F.percentile_approx(col, [0.00, 0.01, 0.25, 0.50, 0.75, 0.99, 1.00], 1).alias('percentiles'))
            .collect())

        target_dict = {}
        for target in target_q:
            target_val = target[self.target]
            q00, q01, q25, q50, q75, q99, q100 = target.percentiles
            target_dict[target_val] = {
                    'q00': q00,
                    'q01': q01,
                    'q25': q25,
                    'q50': q50,
                    'q75': q75,
                    'q99': q99,
                    'q100': q100
                }
        quantile_dict[col] = target_dict

    # returns dictionary indexed by column -> target_value -> quantile

    return quantile_dict

Explorer.make_box = make_box


e = Explorer()
self = e

self.target = 'bin_target'

In [105]:
e.make_box(cols)

{'num1': {0: {'q00': 806.5001352447177,
   'q01': 806.5001352447177,
   'q25': 806.5001352447177,
   'q50': 806.5001352447177,
   'q75': 806.5001352447177,
   'q99': 806.5001352447177,
   'q100': 806.5001352447177},
  1: {'q00': 770.9262639045318,
   'q01': 770.9262639045318,
   'q25': 770.9262639045318,
   'q50': 770.9262639045318,
   'q75': 770.9262639045318,
   'q99': 770.9262639045318,
   'q100': 770.9262639045318}}}

In [192]:
# pandas
class PandasExplorer:
    pass
cols = ['num1', 'num2']
# pandas
def make_box(self, cols):
    df_q = df.groupby('bin_target')[cols].quantile((0.00, 0.01, 0.25, 0.50, 0.75, 0.99, 1.00)).reset_index()

    quantile_dict = {}
    for col in cols:
        target_dict = {}
        for target in df_q[self.target].unique():
            target_q = df_q.loc[df_q[self.target] == target, :]
            q00, q01, q25, q50, q75, q99, q100 = target_q[col].values
            target_dict[target] = {
                'q00': q00,
                'q01': q01,
                'q25': q25,
                'q50': q50,
                'q75': q75,
                'q99': q99,
                'q100': q100
            }

        quantile_dict[col] = target_dict

    return quantile_dict

PandasExplorer.make_box = make_box
p = PandasExplorer()
p.target = 'bin_target'
self = p

quantile_dict = p.make_box(cols)

In [274]:
import colorcet
self = p
from plotly.subplots import make_subplots
p.target_vals = [0, 1]


def plot_box(self, quantile_dict):
    if len(quantile_dict.keys()) == 1:
        col = list(quantile_dict.keys())[0]
        fig = go.Figure()
        for idx, target_val in enumerate(self.target_vals):
            fig.add_trace(go.Box(
                q1=[quantile_dict[col][target_val]['q25']],
                median=[quantile_dict[col][target_val]['q50']],
                q3=[quantile_dict[col][target_val]['q75']],
                x=[target_val], 
                upperfence=[quantile_dict[col][target_val]['q99']],
                lowerfence=[quantile_dict[col][target_val]['q01']],
                name=target_val,
                marker=dict(color=colorcet.glasbey_dark[idx])
            ))

        fig.show()

    else:
        num_cols = len(list(quantile_dict.keys()))
        fig = make_subplots(rows=math.ceil(num_cols/3), cols=3, subplot_titles=list(quantile_dict.keys()))
        row_num = 1
        col_num = 1
        for col in quantile_dict.keys():
            for idx, target_val in enumerate(self.target_vals):
                fig.add_trace(go.Box(
                    q1=[quantile_dict[col][target_val]['q25']],
                    median=[quantile_dict[col][target_val]['q50']],
                    q3=[quantile_dict[col][target_val]['q75']],
                    x=[target_val], 
                    upperfence=[quantile_dict[col][target_val]['q99']],
                    lowerfence=[quantile_dict[col][target_val]['q01']],
                    name=target_val,
                    marker=dict(color=colorcet.glasbey_dark[idx])
                ), row=row_num, col=col_num)            
                
            col_num += 1
            if col_num > 3:
                row_num += 1
                col_num = 1

        fig.update_layout(
            autosize=False,
            width=1200,
            height=math.ceil(num_cols/3) * 400
        )
        fig.show()

In [276]:
plot_box(p, quantile_dict)

In [216]:
import math
math.ceil(4/3)

2

# Histograms

In [327]:
dd.columns
cols = ['num1','num2']

In [69]:
#py
def make_hist(self, cols, n_bins=None, bins=None):
    col_dict = {}
    for col in cols:
        target_dict = {}
        for target_val in self.target_vals:
            target_dict[target_val] = {'bins': None, 'edges': None}
            target_dict[target_val]['edges'], target_dict[target_val]['bins']=(
                dd.filter(F.col(self.target_col)==target_val)
                .select(col).rdd.flatMap(lambda x: x).histogram(n_bins if n_bins else bins))

            target_dict[target_val]['edges'] = target_dict[target_val]['edges'][:-1] # off-by-one

        col_dict[col] = target_dict

    return col_dict

class Explorer:
    pass

Explorer.make_hist = make_hist

e = Explorer()
e.target_col = 'bin_target'
e.target_vals = [0, 1]
e.make_hist(cols, bins=[0, 10, 100])

{'num1': {0: {'bins': [0, 0], 'edges': [0, 10]},
  1: {'bins': [0, 0], 'edges': [0, 10]}},
 'num2': {0: {'bins': [0, 5], 'edges': [0, 10]},
  1: {'bins': [5, 19], 'edges': [0, 10]}}}

In [65]:
#pandas
def make_hist(self, cols, n_bins=None, bins=None):
    col_dict = {}
    for col in cols:
        target_dict = {}
        for target_val in self.target_vals:
            target_dict[target_val] = {'bins': None, 'edges': None}
            target_dict[target_val]['bins'], target_dict[target_val]['edges']  = np.histogram(df.loc[df[self.target_col]==target_val, col], bins=n_bins if n_bins else bins)

        col_dict[col] = target_dict

    return col_dict

class PandasExplorer:
    pass

PandasExplorer.make_hist = make_hist
 
p = PandasExplorer()
p.target_vals = (0, 1)
p.target_col = 'bin_target'
p.make_hist(['num1'], n_bins=5)

{'num1': {0: {'bins': array([2, 7, 2, 4, 2]),
   'edges': array([ 803.04761684,  881.89920232,  960.7507878 , 1039.60237328,
          1118.45395876, 1197.30554423])},
  1: {'bins': array([10, 25, 38,  8,  2]),
   'edges': array([ 769.41881392,  877.26238293,  985.10595194, 1092.94952095,
          1200.79308996, 1308.63665897])}}}

In [282]:
def plot_hist(self, hist_dict):
    # single-column
    if len(hist_dict.keys()) == 1:
        col = list(hist_dict.keys())[0]
        fig = go.Figure()
        for idx, target_val in enumerate(self.target_vals):
            fig.add_trace(
                go.Bar(
                    y=hist_dict[col][target_val]['bins'],
                    x=hist_dict[col][target_val]['edges'],
                    name=target_val,
                    marker=dict(color=colorcet.glasbey_dark[idx])
                )
            )
            fig.update_layout(
                autosize=False,
                width=800,
                height=800,
                title=col
            )
    else:
        num_cols = len(list(hist_dict.keys()))
        fig = make_subplots(rows=math.ceil(num_cols/3), cols=3, subplot_titles=list(hist_dict.keys()))
        row_num = 1
        col_num = 1
        for col in hist_dict.keys():
            for idx, target_val in enumerate(self.target_vals):
                fig.add_trace(go.Bar(
                    y=hist_dict[col][target_val]['bins'],
                    x=hist_dict[col][target_val]['edges'],
                    name=target_val,
                    marker=dict(color=colorcet.glasbey_dark[idx])
                ), row=row_num, col=col_num)

            col_num += 1
            if col_num > 3:
                row_num += 1
                col_num = 1

        fig.update_layout(
            autosize=False,
            width=1200,
            height=math.ceil(num_cols/3) * 400
        )

    fig.show()

PandasExplorer.plot_hist = plot_hist
p = PandasExplorer()
p.target_vals = [0, 1]
p.target_col = 'bin_target'
p.make_hist(cols, n_bins=10)

{'num1': {0: {'bins': array([2, 2, 2, 1, 2, 7, 2, 2, 2, 4]),
   'edges': array([ 806.50013524,  839.03743564,  871.57473604,  904.11203644,
           936.64933684,  969.18663723, 1001.72393763, 1034.26123803,
          1066.79853843, 1099.33583882, 1131.87313922])},
  1: {'bins': array([ 2,  2,  9, 11, 16, 18,  8,  6,  1,  1]),
   'edges': array([ 770.9262639 ,  820.14576102,  869.36525813,  918.58475524,
           967.80425235, 1017.02374946, 1066.24324657, 1115.46274368,
          1164.6822408 , 1213.90173791, 1263.12123502])}},
 'num2': {0: {'bins': array([1, 6, 4, 5, 2, 2, 2, 3, 0, 1]),
   'edges': array([-162.77491541, -120.94079813,  -79.10668085,  -37.27256356,
             4.56155372,   46.39567101,   88.22978829,  130.06390557,
           171.89802286,  213.73214014,  255.56625742])},
  1: {'bins': array([ 2,  9, 10, 21, 11, 10,  7,  2,  1,  1]),
   'edges': array([-220.38681426, -168.21843328, -116.05005231,  -63.88167133,
           -11.71329036,   40.45509062,   92.623471

In [283]:
p.plot_hist(p.make_hist(cols, n_bins=10))

# Trend/Time-Series

## DateUtils


In [279]:
date_cols = 'date'
not isinstance(date_cols, str)

False

In [311]:
import logging

class DateFormatError(Exception):
    def __init__(self, message:str=''):
        self.message = message
        super().__init__(self.message)
# need to assume that we can get a year-month axis out of whatever dates we have
# pyspark
def format_dates(self, date_cols):
    '''Takes dataframe and ensures a consistent year_month time axis'''
    if self.date_format == 'list' and len(date_cols) == 2:
        # assume year/month
        if 'Year' in date_cols and 'Month' in date_cols:
            self.dd = self.dd.withColumn('year_month', F.expr('make_date(Year, Month, 1)'))
        elif 'year' in date_cols and 'month' in date_cols:
            self.dd = self.dd.withColumn('year_month', F.expr('make_date(Year, Month, 1)'))
        else:
            raise DateFormatError(f'Date column format {date_cols} not recognized')
    elif self.date_format == 'american':
        if type(date_cols) is not str:
            raise DateFormatError(f'If date_format=american, type of date_cols must be string, type of date_cols is {type(date_cols)}')
        # dd/MM/yyyy
        self.dd = self.dd.withColumn('year_month', F.to_date(self.date_cols, 'dd/MM/yyyy'))
        self.dd = self.dd.withColumn('year_month', F.expr('make_date(Year(year_month), Month(year_month), Day(year_month)'))
        
    elif self.date_format == 'julian':
        # yyyy/DD
        if type(date_cols) is not str:
            raise DateFormatError(f'If date_format=american, type of date_cols must be string, type of date_cols is {type(date_cols)}')
        self.dd = self.dd.withColumn('year_month', F.to_date(self.date_cols, 'yyyy/DDD'))
        self.dd = self.dd.withColumn('year_month', F.expr('make_date(Year(year_month), Month(year_month), Day(year_month)'))
    else:
        try:
            self.dd = self.dd.withColumn('year_month', F.to_date(date_cols, self.date_format))
            self.dd = self.dd.withColumn('year_month', F.expr('make_date(Year(year_month), Month(year_month), 1'))
        except Exception as e:
            logging.error(f'Date formatting failed: {e}')

    return self.dd


def make_ts(self, cols):
    pass

class Explorer: pass

Explorer.format_dates = format_dates
e = Explorer()
e.target_col = 'bin_target'
e.date_cols = ['year', 'month']
# need to specify date-format, 'list', 'american', 'julian'
e.date_format = 'list' 
e.dd = dd

In [416]:
# pandas
def format_dates(self, date_cols):
    '''Takes dataframe and ensures a consistent year_month time axis'''
    try:
        if self.date_format == 'list' and len(date_cols) == 2:
            # assume year/month
            if 'Year' in date_cols and 'Month' in date_cols:
                self.df['year_month'] = pd.to_datetime(dict(year=self.df['Year'], month=self.df['Month'], day=1))
            elif 'year' in date_cols and 'month' in date_cols:
                self.df['year_month'] = pd.to_datetime(dict(year=self.df['year'], month=self.df['month'], day=1))
            else:
                raise DateFormatError(f'Date column format {date_cols} not recognized')

        elif self.date_format == 'american':
            if type(date_cols) is not str:
                raise DateFormatError(f'If date_format=american, type of date_cols must be string, type of date_cols is {type(date_cols)}')
            # dd/MM/yyyy
            self.df['year_month'] = pd.to_datetime(self.df[date_cols], format='%d-%m-%y')
            
        elif self.date_format == 'julian':
            # yyyy/DD
            if type(date_cols) is not str:
                raise DateFormatError(f'If date_format=american, type of date_cols must be string, type of date_cols is {type(date_cols)}')
            self.df['year_month'] = pd.to_datetime(self.df[date_cols], format='%Y%j')
    except Exception as e:
        logging.error('Manual date formatting failed')
        try:
            self.df['year_month'] = pd.to_datetime(self.df[date_cols], errors='raise')
        except Exception as e:
            logging.error(f'Date formatting failed: {e}')
            return
        else:
            return self.df
    else:
        return self.df

class PandasExplorer: pass
PandasExplorer.format_dates = format_dates
p = PandasExplorer()
p.date_cols = 'american_date' 
p.date_format = 'julian'
p.df = df


In [417]:
df['julian_date'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=1)).dt.strftime('%Y%j')
df['american_date'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=1)).dt.strftime('%d-%m-%Y')
df.head()

/tmp/ipykernel_33386/3847663805.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_33386/3847663805.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,year,month,num1,num2,cat1,bin_target,multi_target,julian_date,american_date,year_month
0,A,2023,1.0,1038.700744,-124.516033,c,1,-1,2023001,01-01-2023,2023-01-01
1,A,2023,2.0,1080.922140,-116.139286,b,1,0,2023032,01-02-2023,2023-01-02
2,A,2023,3.0,1128.626480,301.296995,c,1,0,2023060,01-03-2023,2023-01-03
3,A,2023,4.0,978.441120,-80.501943,b,0,-1,2023091,01-04-2023,2023-01-04
4,A,2023,5.0,999.998574,-146.217577,b,1,1,2023121,01-05-2023,2023-01-05


In [502]:
from typing import Union, List
import pyspark.sql.functions as F
from pandas import to_datetime

class DateFormatError(Exception):
    def __init__(self, message:str=''):
        self.message = message
        super().__init__(self.message)

class SparkDateUtil(object):
    """Used to create a consistent year-month column in pyspark dataframes

    Supported arguments to create_yearmonth are:
        df: a pyspark dataframe
        date_cols: either a list ('year', 'month') 
        format: a format specified in _DATE_FORMAT_MAP OR 'list' if date_cols is a list 

    Example usage:
        df = # pyspark dataframe
        df = SparkDateUtil.create_yearmonth(df, date_cols=['year', 'month'], format='list')

    Raises:
        DateFormatError: if the number of date-columns is more than 3
        ValueError: thrown from the spark to_date method 
    
    """
    _DATE_FORMAT_MAP = {
        'american': '%m%d%y', 
        'julian': '%Y%j', 
        'iso': '%y%m%d', 
        'american_dash': '%m-%d-%y', 
        'american_fwslash': '%m/%d/%y',
        'julian_dash': '%Y-%j',
        'julian_fwslash': '%Y/%j',
        'iso_dash': '%y-%m-%d',
        'iso_fwslash': '%y/%m/%d'
    }
    @staticmethod
    def create_yearmonth(df, date_cols:Union[List, str], format: str):
        if format == 'list':
            if len(date_cols) > 3:
                raise DateFormatError('Year, Month and Day alone supported')
            if len(date_cols) == 2:
                # year and month OR Year and Month
                if 'year' in date_cols and 'month' in date_cols:
                    df = SparkDateUtil._create_spark_yearmonth_from_list(df, date_cols)
                if 'Year' in date_cols and 'Month' in date_cols:
                    df = SparkDateUtil._create_spark_yearmonth_from_list(df, date_cols)

        else:
            df = df.withColumn('year_month', F.to_date(F.col(date_cols), format=SparkDateUtil._DATE_FORMAT_MAP[format]))


        return df

    @staticmethod
    def _create_spark_yearmonth_from_list(df, date_cols):
        if len(date_cols) == 2:
            return df.withColumn('year_month', F.expr(f'make_date({date_cols[0]}, {date_cols[1]}, 1)'))
        elif len(date_cols) == 3: 
            return df.withColumn('year_month', F.expr(f'make_date({date_cols[0]}, {date_cols[1]}, {date_cols[2]})'))

class PandasDateUtil(object):
    """Used to create a consistent year-month column in pandas dataframes

    Supported arguments to create_yearmonth are:
        df: a Pandas dataframe
        date_cols: either a list ('year', 'month') 
        format: a format specified in _DATE_FORMAT_MAP OR 'list' if date_cols is a list 

    Example usage:
        df = # pandas dataframe
        df = PandasDateUtil.create_yearmonth(df, date_cols=['year', 'month'], format='list')

    Raises:
        DateFormatError: if the number of date-columns is more than 3
        ValueError: thrown from pandas in the to_datetime method
    
    """
    _DATE_FORMAT_MAP = {
        'american': '%m%d%Y', 
        'julian': '%Y%j', 
        'iso': '%Y%m%d', 
        'american_dash': '%m-%d-%Y', 
        'american_fwslash': '%m/%d/%Y',
        'julian_dash': '%Y-%j',
        'julian_fwslash': '%Y/%j',
        'iso_dash': '%Y-%m-%d',
        'iso_fwslash': '%Y/%m/%d'
    }
    @staticmethod
    def create_yearmonth(df, date_cols:Union[List, str], format: str):
        if format == 'list':
            if len(date_cols) > 3:
                raise DateFormatError('Year, Month and Day alone supported')
            if len(date_cols) == 2:
                # year and month OR Year and Month
                if 'year' in date_cols and 'month' in date_cols:
                    return df.assign(year_month=to_datetime(dict(
                        year=df.year,
                        month=df.month,
                        day=1
                    )))
                elif 'Year' in date_cols and 'Month' in date_cols:
                    return df.assign(year_month=to_datetime(dict(
                        year=df.Year,
                        month=df.month,
                        day=1
                    )))
        else:
            if PandasDateUtil._DATE_FORMAT_MAP.get(format, None) is not None:

                logging.debug(PandasDateUtil._DATE_FORMAT_MAP[format])
                return df.assign(year_month=to_datetime(df[date_cols], format=PandasDateUtil._DATE_FORMAT_MAP.get(format, None)))
            else:
                return df.assign(year_month=to_datetime(df[date_cols]))


<class '__main__.PandasDateUtil'>


In [496]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
PandasDateUtil.create_yearmonth(df, 'american_date', format='american_dash')

DEBUG:root:%m-%d-%Y


,id,year,month,num1,num2,cat1,bin_target,multi_target,julian_date,american_date,year_month
0,A,2023,1.0,1038.700744,-124.516033,c,1,-1,2023001,01-01-2023,2023-01-01
1,A,2023,2.0,1080.922140,-116.139286,b,1,0,2023032,01-02-2023,2023-01-02
2,A,2023,3.0,1128.626480,301.296995,c,1,0,2023060,01-03-2023,2023-01-03
3,A,2023,4.0,978.441120,-80.501943,b,0,-1,2023091,01-04-2023,2023-01-04
4,A,2023,5.0,999.998574,-146.217577,b,1,1,2023121,01-05-2023,2023-01-05
5,A,2023,6.0,1078.113199,-28.030055,a,1,0,2023152,01-06-2023,2023-01-06
6,A,2023,7.0,1139.008626,31.238760,a,1,1,2023182,01-07-2023,2023-01-07
7,A,2023,8.0,1013.242755,28.849862,c,1,0,2023213,01-08-2023,2023-01-08
8,A,2023,9.0,821.242266,126.548980,c,1,0,2023244,01-09-2023,2023-01-09
9,A,2023,10.0,1003.563961,137.037371,c,1,0,2023274,01-10-2023,2023-01-10


In [533]:
# pyspark
def plot_trend(self, cols):
    if 'year_month' not in self.dd.columns:
        raise KeyError('year_month must be in dataframe') 

    # mean aggs
    mean_agg = [F.mean(col).alias(f'mean_col') for col in cols]
    max_agg = [F.max(col).alias(f'max_col') for col in cols]
    min_agg = [F.min(col).alias(f'min_col') for col in cols]
    std_agg = [F.stddev(col).alias(f'stddev_col') for col in cols]
    trend = self.dd.groupBy([self.target, 'year_month']).agg(*[*mean_agg, *max_agg, *min_agg, *std_agg]).toPandas()
    for target_val in trend[self.target].unique():
        # plot here

class Explorer: pass
Explorer.plot_trend = plot_trend
e = Explorer()
e.target = 'bin_target'
e.dd = dd

In [534]:
logger.setLevel(logging.WARN)
e.plot_trend(['num1'])

,bin_target,year_month,mean_col,max_col,min_col,min_col
0,1,2023-07-01,1139.008626,1139.008626,1139.008626,NaN
1,1,2023-01-01,1038.700744,1038.700744,1038.700744,NaN
2,1,2023-08-01,1013.242755,1013.242755,1013.242755,NaN
3,1,2023-06-01,1078.113199,1078.113199,1078.113199,NaN
4,1,2023-02-01,1080.922140,1080.922140,1080.922140,NaN
5,1,2023-03-01,1128.626480,1128.626480,1128.626480,NaN
6,1,2023-05-01,999.998574,999.998574,999.998574,NaN
7,0,2023-04-01,978.441120,978.441120,978.441120,NaN
8,1,2023-11-01,982.852194,982.852194,982.852194,NaN
9,1,None,1000.377346,1263.121235,770.926264,90.336798


# Correlation

In [69]:
from pyspark.sql.types import StructType, FloatType, DoubleType, StructField
# if pyspark default to datashader

df = pd.DataFrame({
    'num1': np.random.normal(1000, 10, size=100000), 
    'bin_target': np.random.choice([0, 1], size=100000)
})

df['num2'] = df.num1 ** 2

dd = spark.createDataFrame(df)

/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [123]:
import logging
import colorcet
import plotly.graph_objects as go
from plotly import express as px

# pyspark
def plot_corr_scatter(self, col1, col2):
    corr = self.dd.select(*[col1, col2]).stat.corr('num1', 'num2')
    data = self.dd.select(*[col1, col2, self.target]).toPandas()
    self._plot_corr(col1, col2, data)

# pandas 
# def plot_corr(self, col1, col2):
#     _plot_corr(col1, col2, self.df)
    

def _plot_corr_scatter(self, col1, col2, data):
    if len(data) > 10000:
        logging.warning('Sampling done, data too large')
        data = data.sample(10000)
    
    fig = px.scatter(
        data, col1, col2,
        marginal_x='histogram',
        marginal_y='histogram',
        color=df[self.target] # HACK need to set target to astype(category)
    ) 

    fig.update_layout(
        autosize=False,
        width=800,
        height=800,
        xaxis=dict(title=col1),
        yaxis=dict(title=col2),
        title=f'{col2} against {col1}'
    )
    fig.show()


class Explorer: pass

Explorer.plot_corr = plot_corr
Explorer._plot_corr = _plot_corr
e = Explorer()
e.target = 'bin_target'
e.target_vals = [0, 1]
e.dd = dd

In [124]:
e.plot_corr(col1, col2)

# Bar Chart for Categorical

In [236]:
from typing import Union, List
# pyspark
def make_bar(self, cols: Union[List, str]):
    grouped = dd.groupBy([self.target, *cols]).count().toPandas()
    print(grouped)
    fig = go.Figure()
    for col in cols:
        for target_val in grouped[self.target].unique():
            fig.add_trace(go.Bar(
                x=grouped[col].unique(),
                y=grouped.loc[grouped[self.target]==target_val, 'count'],
                name=f'Target: {target_val}'
            ))

    fig.show()

# pandas
def make_bar(self, col: Union[List, str]):
    self.df = self.df.assign(count=1)
    grouped = self.df.groupby([self.target, col], as_index=False).count()
    fig = go.Figure()
    for target_val in grouped[self.target].unique():
        fig.add_trace(go.Bar(
            x=grouped[col].unique(),
            y=grouped.loc[grouped[self.target]==target_val, 'count'],
            name=f'Target: {target_val}'
        ))

    self.df.drop('_cnt', axis=1, inplace=True)
    fig.show()

class SE:pass
SE.make_bar = make_bar
s = SE()
s.target='bin_target'
s.make_bar(['cat1'])


   bin_target cat1  count
0           1    c     32
1           1    a     26
2           0    a      7
3           1    b     25
4           0    c      6
5           0    b      4


In [111]:
df.groupby(['bin_target', 'cat1'], as_index=False).count()

,bin_target,cat1,id,year,month,num1,num2,cat2,multi_target
0,0,a,9,9,9,9,9,9,9
1,0,b,6,6,6,6,6,6,6
2,0,c,7,7,7,7,7,7,7
3,1,a,30,30,30,30,30,30,30
4,1,b,26,26,26,26,26,26,26
5,1,c,22,22,22,22,22,22,22


In [116]:
df['bin_target'] = df['bin_target'].astype('category')
px.scatter(
    df, col1, col2,
    marginal_x='histogram',
    marginal_y='histogram',
    color=self.target
)

# Heatmap Correlation

In [143]:
import plotly.io as pio

def plot_corr_map(self, cols:list):
    import pandas
    import pyspark
    pio.templates.default = 'plotly_white'
    if isinstance(self.data, pyspark.sql.DataFrame):
        data = self.data[cols].toPandas()
    else:
        data = self.data[cols]

    corr = data.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    rLY = corr.mask(mask)

    fig = go.Figure(
        data=go.Heatmap(
            z=corr.mask(mask),
            x=corr.columns,
            y=corr.columns,
            colorscale=colorcet.coolwarm,
            zmin=-1,
            zmax=1
        ),
        layout=go.Layout(
            autosize=False,
            height=800 if len(corr.columns) < 4 else len(corr.columns) * 100,
            width=800 if len(corr.columns) < 4 else len(corr.columns) * 100,
        ) 
    )
    fig.show() 

class Explorer: pass
Explorer.plot_corr_map = plot_corr_map
e = Explorer()
e.data = dd

In [144]:
e.plot_corr_map(['num1', 'num2'])

# Missing Values

In [172]:
from pyspark.sql import functions as F

N = 10000
df = pd.DataFrame({
    'num1': np.random.normal(100, 10, size=N),
    'num2': np.random.normal(100, 10, size=N),
    'cat1': np.random.choice(('a', 'b', 'c', None), size=N),
    'cat2': np.random.choice(('a', 'b', 'c', None), size=N),
})

dd = spark.createDataFrame(df)

perc_missing = dd.select([(F.count(F.when(F.isnan(c) | F.col(c).isNull(), c))/F.count(F.lit(1))).alias(c) for c in dd.columns]).toPandas()
perc_missing

/home/aadi/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



,num1,num2,cat1,cat2
0,0.0,0.0,0.2584,0.2525


In [165]:
perc_missing.values


array([[0.    , 0.    , 0.2546, 0.2451]])

In [166]:
df = df.iloc[:, [i for i, n in enumerate(np.var(df.isnull(), axis='rows')) if n > 0]]
corr_mat = df.isnull().corr()

In [173]:
df

,num1,num2,cat1,cat2
0,121.083356,89.056597,None,a
1,99.497182,91.558549,a,c
2,93.158829,96.360426,c,c
3,104.238097,106.300758,b,a
4,82.883019,101.571009,a,b
...,...,...,...,...
9995,101.635382,104.079928,b,b
9996,90.963823,87.152202,a,c
9997,97.515792,104.485899,None,a
9998,97.613713,97.051310,None,c


In [174]:
df.iloc[:, np.argsort(df.count(axis='rows').values)]

,cat1,cat2,num1,num2
0,None,a,121.083356,89.056597
1,a,c,99.497182,91.558549
2,c,c,93.158829,96.360426
3,b,a,104.238097,106.300758
4,a,b,82.883019,101.571009
...,...,...,...,...
9995,b,b,101.635382,104.079928
9996,a,c,90.963823,87.152202
9997,None,a,97.515792,104.485899
9998,None,c,97.613713,97.051310


In [200]:
df.count(axis='rows')

num1    10000
num2    10000
cat1     7416
cat2     7475
dtype: int64

In [205]:
counts = dd.select(*[F.count(col).alias(col) for col in dd.columns]).toPandas()

counts.index = ['cnt']
cols = counts.columns
vals = counts.values[0]

series = pd.Series(data=vals, index=cols)

In [212]:

np.argsort(series).values
new_col_order = []
for col_id in np.argsort(series).values:
    new_col_order.append(dd.columns[col_id])

In [213]:
dd.select(*new_col_order)

DataFrame[cat1: string, cat2: string, num1: double, num2: double]

In [283]:
def null_sort(data, sort=None):
    if sort is None:
        return data

    if sort not in ['ascending', 'descending']:
        raise ValueError('sort must be on of "ascending" or "descending"')

    counts = data.select(*[F.count(col).alias(col) for col in data.columns]).toPandas()

    counts.index = ['cnt']
    cols = counts.columns
    vals = counts.values[0]

    series = pd.Series(data=vals, index=cols)
    new_col_order = []
    if sort == 'ascending': 
        for col_id in np.argsort(series).values:
            new_col_order.append(data.columns[col_id])

        return data.select(*new_col_order)

    elif sort == 'descending':
        for col_id in np.flipud(np.argsort(series.values)):
            new_col_order.append(data.columns[col_id])

        return data.select(*new_col_order)
    

In [219]:
filter = 'top'
if filter == 'top':
    if p:
        df = df.iloc[:, [c >= p for c in df.count(axis='rows').values / len(df)]]
    if n:
        df = df.iloc[:, np.sort(np.argsort(df.count(axis='rows').values)[-n:])]
elif filter == 'bottom':
    if p:
        df = df.iloc[:, [c <= p for c in df.count(axis='rows').values / len(df)]]
    if n:
        df = df.iloc[:, np.sort(np.argsort(df.count(axis='rows').values)[:n])]
return df


In [225]:
data =dd
count_rows_values = data.select(*[F.count(col).alias(col) for col in data.columns]).toPandas().values[0]

[]

array([10000, 10000,  7416,  7475])

In [235]:
def null_filter(data, _filter=None, p=0, n=0):
    new_col_order = data.columns
    if _filter == 'top':
        new_col_order = []
        len_df = data.count()
        if p:
            for idx,col in enumerate(data.columns):
                if count_rows_values[idx] / len_df >= p:
                    new_col_order.append(col)

        if n:
            raise NotImplementedError

    elif _filter == 'bottom':
        new_col_order = []
        len_df = data.count()
        if p:
            for idx,col in enumerate(data.columns):
                if count_rows_values[idx] / len_df <= p:
                    new_col_order.append(col)

        if n:
            raise NotImplementedError

    return data.select(*new_col_order)


In [236]:
null_filter(dd, _filter='top', p=0.25).show()

+------------------+------------------+----+----+
|              num1|              num2|cat1|cat2|
+------------------+------------------+----+----+
|121.08335611403442| 89.05659728013157|null|   a|
| 99.49718236827349| 91.55854914028446|   a|   c|
| 93.15882854556602| 96.36042629671182|   c|   c|
|104.23809744813724|106.30075816945659|   b|   a|
|  82.8830192444044| 101.5710090562014|   a|   b|
|103.69121169839384|  96.7092705345947|   c|null|
|102.37999678904094| 87.14747874712171|   b|   a|
|100.90273922845884|107.52146619365676|   c|   a|
|100.60988914543833| 93.28157252048585|null|null|
| 93.60497092422322|109.22049717666384|   a|null|
|100.97353196353681|103.86161678910045|   a|   a|
|100.93328277290303| 79.86499279739459|null|   c|
| 97.15614891641243|122.31310035629119|   c|   b|
| 103.0227334660734| 97.20419009392349|   b|   c|
|106.54771910975283| 91.90441529544518|   a|   b|
| 84.68103658702624|114.68983029012036|null|null|
|104.16292660423326|116.97725330695381|null|   b|


In [239]:
dd = null_filter(dd, _filter=None)
dd = nullity_sort(dd, sort=None)

In [249]:
# remove completely full or emptry values


In [253]:
col_counts = count_rows_values(data)


In [261]:
len_df = data.count()
col_counts[(col_counts != 0) & (col_counts != len_df)]

data.select(*col_counts.index)

DataFrame[num1: double, num2: double, cat1: string, cat2: string]

In [246]:
dd.num1.isNull().show()

TypeError: 'Column' object is not callable

In [248]:
import pyspark

pyspark.pandas

AttributeError: module 'pyspark' has no attribute 'pandas'

In [289]:
def count_rows_values(data):
    counts = data.select(*[F.count(col).alias(col) for col in data.columns]).toPandas().values[0]

    return pd.Series(index=data.columns, data=counts)

def plot_heatmap_missing(data):
    data = null_filter(data)
    data = null_sort(data)

    col_counts = count_rows_values(data)

    if isinstance(data, pyspark.sql.DataFrame):
        data = data.toPandas()

    data_small = data[col_counts.index]
    corr_mat = data_small.isnull().corr()
    mask = np.zeros_like(corr_mat)
    mask[np.triu_indices_from(mask)] = True

    fig = go.Figure(go.Heatmap(
        z=corr_mat.values,
        x=corr_mat.columns,
        y=corr_mat.columns,
        zmin=-1,
        zmax=1, 
        colorscale=colorcet.coolwarm
    ), layout=go.Layout(
        autosize=False,
        width=1200,
        height=1200,
        title='Missing Value Correlations'
    ))

    fig.show()

In [290]:
plot_heatmap_missing(data)

In [266]:
corr_mat = data_small.toPandas().isnull().corr()
mask = np.zeros_like(corr_mat)
mask[np.triu_indices_from(mask)] = True

In [275]:
from plotly.graph_objects import Heatmap

fig = go.Figure(go.Heatmap(
    z=corr_mat.values,
    x=corr_mat.columns,
    y=corr_mat.columns,
    zmin=-1,
    zmax=1, 
    colorscale=colorcet.coolwarm
), layout=go.Layout(
    autosize=False,
    width=1200,
    height=1200,
    title='Missing Value Correlations'
))

In [276]:
fig.show()

In [293]:
import sys

sys.path.append('/storage/projects/notes/machine-learning')

In [295]:
import edaclass

# Class

## Pyspark

In [491]:
import logging
import math
from abc import ABC, abstractmethod, ABCMeta
from typing import Dict, List, Union

import colorcet
from pandas import to_datetime
from plotly import express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from pyspark.sql import functions as F


class Explorer(metaclass=ABCMeta):
    '''Abstract base class used for EDA
    
    Methods that must be overridden are:
        make_hist
        make_box
        make_bar
        null_sort
        null_filter

        *see method docstrings for functionality and output signatures 
    '''

    @abstractmethod
    def make_bar(self, col: Union[List, str]):
        '''Makes grouping-count for each column
        
        Args:
            col: list/name of columns to count for

        Returns:
            groupby pandas dataframe
        '''
        pass

    @abstractmethod
    def make_box(self, cols: Union[List, str]) -> Dict:
        '''Generates quartiles for box-whisker plotting
        

        Args:
            cols: list/string of column(s) to plot

        Output must be a dictionary indexed by:
        column
            | target_value
                | quantile
            
            *if target_values are categorical

        OR
        column
            | quantile

        Example output for binary target:
         {
            "num1": {
                "0": {
                    "q00": 803.0476168393415,
                    "q01": 803.0476168393415,
                    "q25": 907.2920695654886,
                    "q50": 959.0112145260407,
                    "q75": 1059.4072543931134,
                    "q99": 1197.3055442349162,
                    "q100": 1197.3055442349162
                },
                "1": {
                    "q00": 769.4188139185309,
                    "q01": 769.4188139185309,
                    "q25": 932.2965098911496,
                    "q50": 1003.9672686831474,
                    "q75": 1048.7928364881604,
                    "q99": 1308.6366589738643,
                    "q100": 1308.6366589738643
                }
            },
        }

        Quantiles for 0.00, 0.01, 0.25, 0.50, 0.75, 0.99 and 1.00 must be specified 

        Returns: 
            quantile_dict
        
        '''
        pass

    @abstractmethod
    def make_hist(self, cols: Union[List, str], bins=None, n_bins=None) -> Dict:
        '''Generates histogram for each column, split by target if type categorical/binary
        
        Args:
            cols: list/string of column(s) to generate histograms for
            bins: list of bin-edges
            n_bins: number of bins for histogram

        Raises:
            AttributeError if both bins and n_bins are defined

        Returns:
            dictionary of (bins, edges) indexed by column name
        '''
        pass

    @abstractmethod
    def null_filter(self, data, _filter: str, p: float) -> pyspark.sql.DataFrame:
        '''Filters columns with a percentage p missing or less'''
        pass

    @abstractmethod
    def null_sort(self, data, sort) -> pyspark.sql.DataFrame:
        '''Sorts columns by percentage missing values'''
        pass

    @abstractmethod
    def plot_heatmap_missing(self, cols:list) -> None:
        """Plots a heatmap of correlation of missing values
        
        Args:
            cols: list of columns to plot missing correlation, if not specified, uses full dataset 
        
        """
        pass

    def _validate_target_type(self, target):
        '''Helper method to ensure target is one of "binary", "categorical" or "continuous"'''
        if target not in ('binary', 'categorical', 'continuous'):
            raise ValueError(f'target must be one of "binary", "categorical" or "continuous", current target is {target}')

    def plot_hist(self, cols: Union[List, str], bins=None, n_bins=None):
        '''Plots histogram using output of make_hist function
        
        Args:
            cols: list/name of column to show histogram for
            bins: list of bin edges
            n_bins: number of bins

        Raises:
            AttributeError if both bins and n_bins are specified
        
        '''

        if bins is not None and n_bins is not None:
            raise AttributeError('Specify either bins or n_bins, not both')

        if isinstance(cols, str):
            cols = [cols]

        hist_dict = self.make_hist(cols=cols, bins=bins, n_bins=n_bins)
        # single-column
        if len(hist_dict.keys()) == 1:
            col = list(hist_dict.keys())[0]
            fig = go.Figure()
            for idx, target_val in enumerate(self.target_vals):
                fig.add_trace(
                    go.Bar(
                        y=hist_dict[col][target_val]["bins"],
                        x=hist_dict[col][target_val]["edges"],
                        name=f'Target: {target_val}',
                        marker=dict(color=colorcet.glasbey_dark[idx]),
                    )
                )
                fig.update_layout(autosize=False, width=800, height=800, title=col)
        else:
            num_cols = len(list(hist_dict.keys()))
            fig = make_subplots(
                rows=math.ceil(num_cols / 3),
                cols=3,
                subplot_titles=list(hist_dict.keys()),
            )
            row_num = 1
            col_num = 1
            for col in hist_dict.keys():
                for idx, target_val in enumerate(self.target_vals):
                    fig.add_trace(
                        go.Bar(
                            y=hist_dict[col][target_val]["bins"],
                            x=hist_dict[col][target_val]["edges"],
                            name=f'Target val: {target_val}',
                            marker=dict(color=colorcet.glasbey_dark[idx]),
                        ),
                        row=row_num,
                        col=col_num,
                    )

                col_num += 1
                if col_num > 3:
                    row_num += 1
                    col_num = 1

            fig.update_layout(
                autosize=False, width=1200, height=math.ceil(num_cols / 3) * 400
            )

        fig.show()

    def plot_box(self, cols: Union[List, str]=None):
        '''Plots quartiles using the make_box function
        
        Args:
            cols: list of columns to plot for, if not specified defaults to all
        
        '''

        if not cols:
            cols = self.data.columns

        quantile_dict = self.make_box(cols)
        if self.target_type in ('binary', 'categorical'):
            if len(quantile_dict.keys()) == 1:
                col = list(quantile_dict.keys())[0]
                fig = go.Figure()
                for idx, target_val in enumerate(self.target_vals):
                    fig.add_trace(
                        go.Box(
                            q1=[quantile_dict[col][target_val]["q25"]],
                            median=[quantile_dict[col][target_val]["q50"]],
                            q3=[quantile_dict[col][target_val]["q75"]],
                            x=[target_val],
                            upperfence=[quantile_dict[col][target_val]["q99"]],
                            lowerfence=[quantile_dict[col][target_val]["q01"]],
                            name=f'Target: {target_val}',
                            marker=dict(color=colorcet.glasbey_dark[idx]),
                        )
                    )
                fig.update_layout(
                    autosize=False, width=800, height=800,
                    title=col
                )
                fig.show()

            else:
                num_cols = len(list(quantile_dict.keys()))
                fig = make_subplots(
                    rows=math.ceil(num_cols / 3),
                    cols=3,
                    subplot_titles=list(quantile_dict.keys()),
                )
                row_num = 1
                col_num = 1
                for col in quantile_dict.keys():
                    for idx, target_val in enumerate(self.target_vals):
                        fig.add_trace(
                            go.Box(
                                q1=[quantile_dict[col][target_val]["q25"]],
                                median=[quantile_dict[col][target_val]["q50"]],
                                q3=[quantile_dict[col][target_val]["q75"]],
                                x=[target_val],
                                upperfence=[quantile_dict[col][target_val]["q99"]],
                                lowerfence=[quantile_dict[col][target_val]["q01"]],
                                name=f'Target: {target_val}',
                                marker=dict(color=colorcet.glasbey_dark[idx]),
                            ),
                            row=row_num,
                            col=col_num,
                        )

                    col_num += 1
                    if col_num > 3:
                        row_num += 1
                        col_num = 1

                fig.update_layout(
                    autosize=False, width=1200, height=math.ceil(num_cols / 3) * 400
                )
                fig.show()

        else:
            if len(quantile_dict.keys()) == 1:
                col = list(quantile_dict.keys())[0]
                fig = go.Figure()
                fig.add_trace(
                    go.Box(
                        q1=[quantile_dict[col]["q25"]],
                        median=[quantile_dict[col]["q50"]],
                        q3=[quantile_dict[col]["q75"]],
                        x=[col],
                        upperfence=[quantile_dict[col]["q99"]],
                        lowerfence=[quantile_dict[col]["q01"]],
                        name=f'Column: {col}',
                        marker=dict(color=colorcet.glasbey_dark[0]),
                    )
                )
                fig.update_layout(
                    autosize=False, width=800, height=800,
                    title=col
                )
                fig.show()

            else:
                num_cols = len(list(quantile_dict.keys()))
                fig = make_subplots(
                    rows=math.ceil(num_cols / 3),
                    cols=3,
                    subplot_titles=list(quantile_dict.keys()),
                )
                row_num = 1
                col_num = 1
                for col in quantile_dict.keys():
                    fig.add_trace(
                        go.Box(
                            q1=[quantile_dict[col]["q25"]],
                            median=[quantile_dict[col]["q50"]],
                            q3=[quantile_dict[col]["q75"]],
                            x=[col],
                            upperfence=[quantile_dict[col]["q99"]],
                            lowerfence=[quantile_dict[col]["q01"]],
                            name=f'Column: {col}',
                            marker=dict(color=colorcet.glasbey_dark[0]),
                        ),
                        row=row_num,
                        col=col_num,
                    )

                    col_num += 1
                    if col_num > 3:
                        row_num += 1
                        col_num = 1

                fig.update_layout(
                    autosize=False, width=1200, height=math.ceil(num_cols / 3) * 400
                )
                fig.show()            

    def plot_corr_scatter(self, col1, col2):
        '''Scatter plot between two columns
        
        Args:
            col1: column on X-axis
            col2: column on Y-axis
        
        '''
        corr = self.data.select(*[col1, col2]).stat.corr("num1", "num2")
        data = self.data.select(*[col1, col2, self.target]).toPandas()
        self._plot_corr_scatter(col1, col2, data)

    def _plot_corr_scatter(self, col1, col2, data):
        if len(data) > 10000:
            logging.warning("Sampling done, data too large")
            data = data.sample(10000)

        fig = px.scatter(
            data,
            col1,
            col2,
            marginal_x="histogram",
            marginal_y="histogram",
            color=data[self.target],  # HACK need to set target to astype(category)
        )

        fig.update_layout(
            autosize=False,
            width=800,
            height=800,
            xaxis=dict(title=col1),
            yaxis=dict(title=col2),
            title=f"{col2} against {col1}",
        )
        fig.show()



    def plot_bar(self, cols: Union[List, str]):
        grouped = self.make_bar(cols)

        if self.target_type in ('binary', 'categorical'):
            self._plot_bar_categorical_target(cols, grouped)
        
        if self.target_type in ('continuous'):
            self._plot_bar_continuous_target(cols, grouped)

    def _plot_bar_categorical_target(self, cols, grouped):
        fig = go.Figure()
        for col in cols:
            for target_val in grouped[self.target].unique():
                fig.add_trace(go.Bar(
                    x=grouped[col].unique(),
                    y=grouped.loc[grouped[self.target]==target_val, 'count'],
                    name=f'Target: {target_val}'
                ))
        fig.show()

    def _plot_bar_continuous_target(self, cols, grouped):
        fig = go.Figure()
        for col in cols:
            fig.add_trace(go.Bar(
                x=grouped[col].unique(),
                y=grouped['count'],
            ))

        fig.show()

    def plot_corr_map(self, cols: list = None):

        if not cols:
            cols = self.data.columns
        if isinstance(self.data, pyspark.sql.DataFrame):
            data = self.data[cols].toPandas()
        else:
            data = self.data[cols]

        corr = data.corr()
        mask = np.triu(np.ones_like(corr, dtype=bool))
        rLY = corr.mask(mask)

        fig = go.Figure(
            data=go.Heatmap(
                z=corr.mask(mask),
                x=corr.columns,
                y=corr.columns,
                colorscale=colorcet.coolwarm,
                zmin=-1,
                zmax=1,
            ),
            layout=go.Layout(
                autosize=False,
                height=800 if len(corr.columns) < 4 else len(corr.columns) * 100,
                width=800 if len(corr.columns) < 4 else len(corr.columns) * 100,
            ),
        )
        fig.show()



In [498]:
import pyspark  

class SparkExplorer(Explorer, ):
    def __init__(self, 
        data: pyspark.sql.DataFrame,
        target: str,
        target_type: str,
        date_cols:Union[List, str] = None,
        date_format: str = None, 
    ):
        '''Spark-specific implementation of Explorer


        Args:
            data: pyspark dataframe
            target: name of target column
            target_type: one of "continuous", "categorical" or "binary"
            date_cols: either a list of ['year', 'month'] OR a single column
            date_format: one of "american", "julian" or "iso" with the dash/dot variations
        
        Raises:
            ValueError: if target-type is not valid

        '''

        self._validate_target_type(target_type)
        self.target_vals = None

        if date_cols and date_format:
            data = SparkDateUtil.create_yearmonth(data, date_cols=date_cols, format=date_format)

        if target_type in ('binary', 'categorical'):
            self.target_vals = self._get_target_vals(data, target)

        self.data = data
        self.target = target
        self.target_type = target_type

    def _get_target_vals(self, data, target):
        return list(data.select(target).distinct().toPandas()[target].values)

    def make_bar(self, cols):
        '''Makes grouping-count for each column
        
        Args:
            col: list/name of columns to count for

        Returns:
            groupby pandas dataframe
        '''
        if self.target_type in ('binary', 'categorical'):
            return self.data.groupBy(*[self.target, *cols]).count().toPandas()
        else:
            return self.data.groupby(*cols).count().toPandas()
        
    def make_box(self, cols):
        '''Generates quartiles for box-whisker plotting
        

        Args:
            cols: list/string of column(s) to plot

        Output must be a dictionary indexed by:
        column
            | target_value
                | quantile
            
            *if target_values are categorical

        OR
        column
            | quantile

        Example output for binary target:
         {
            "num1": {
                "0": {
                    "q00": 803.0476168393415,
                    "q01": 803.0476168393415,
                    "q25": 907.2920695654886,
                    "q50": 959.0112145260407,
                    "q75": 1059.4072543931134,
                    "q99": 1197.3055442349162,
                    "q100": 1197.3055442349162
                },
                "1": {
                    "q00": 769.4188139185309,
                    "q01": 769.4188139185309,
                    "q25": 932.2965098911496,
                    "q50": 1003.9672686831474,
                    "q75": 1048.7928364881604,
                    "q99": 1308.6366589738643,
                    "q100": 1308.6366589738643
                }
            },
        }

        Quantiles for 0.00, 0.01, 0.25, 0.50, 0.75, 0.99 and 1.00 must be specified 

        Returns: 
            quantile_dict
        
        '''
        if self.target_type in ('binary', 'categorical'):
            return self._make_box_categorical_target(cols)
        else:
            return self._make_box_continuous_target(cols)


    def _make_box_categorical_target(self, cols):
        quantile_dict = {}
        for col in cols:
            # q00, q01, q25, q50, q75, q99, q100 = 
            target_q = (self.data.groupBy(self.target)
                .agg(
                    F.percentile_approx(col, [0.00, 0.01, 0.25, 0.50, 0.75, 0.99, 1.00], 1000).alias('percentiles'))
                .collect())

            target_dict = {}
            for target in target_q:
                target_val = target[self.target]
                q00, q01, q25, q50, q75, q99, q100 = target.percentiles
                target_dict[target_val] = {
                        'q00': q00,
                        'q01': q01,
                        'q25': q25,
                        'q50': q50,
                        'q75': q75,
                        'q99': q99,
                        'q100': q100
                    }
            quantile_dict[col] = target_dict

        return quantile_dict

    def _make_box_continuous_target(self, cols):
        for col in cols:
            q00, q01, q25, q50, q75, q99, q100 = self.data.select(
                F.percentile_approx('num1', [0.00, 0.01, 0.25, 0.50, 0.75, 0.99, 1.00]).alias('percentiles')).collect()[0].percentiles

            quantile_dict[col] = {
                    'q00': q00,
                    'q01': q01,
                    'q25': q25,
                    'q50': q50,
                    'q75': q75,
                    'q99': q99,
                    'q100': q100
            }

        return quantile_dict

    def make_hist(self, cols, n_bins=None, bins=None):
        '''Generates histogram for each column, split by target if type categorical/binary
        
        Args:
            cols: list/string of column(s) to generate histograms for
            bins: list of bin-edges
            n_bins: number of bins for histogram

        Raises:
            AttributeError if both bins and n_bins are defined

        Returns:
            dictionary of (bins, edges) indexed by column name
        '''
        col_dict = {}
        for col in cols:
            target_dict = {}
            for target_val in self.target_vals:
                target_dict[target_val] = {'bins': None, 'edges': None}
                target_dict[target_val]['edges'], target_dict[target_val]['bins']=(
                    dd.filter(F.col(self.target)==int(target_val))
                    .select(col).rdd.flatMap(lambda x: x).histogram(n_bins if n_bins else bins))

                target_dict[target_val]['edges'] = target_dict[target_val]['edges'][:-1] # off-by-one

            col_dict[col] = target_dict

        return col_dict


    @staticmethod
    def null_sort(data, sort=None):
        '''Sorts columns by percentage missing values'''
        if sort is None:
            return data

        if sort not in ['ascending', 'descending']:
            raise ValueError('sort must be on of "ascending" or "descending"')

        counts = data.select(*[F.count(col).alias(col) for col in data.columns]).toPandas()

        counts.index = ['cnt']
        cols = counts.columns
        vals = counts.values[0]

        series = pd.Series(data=vals, index=cols)
        new_col_order = []
        if sort == 'ascending': 
            for col_id in np.argsort(series).values:
                new_col_order.append(data.columns[col_id])

            return data.select(*new_col_order)

        elif sort == 'descending':
            for col_id in np.flipud(np.argsort(series.values)):
                new_col_order.append(data.columns[col_id])

            return data.select(*new_col_order)

    @staticmethod
    def null_filter(data, _filter=None, p=0, n=0):
        new_col_order = data.columns
        if _filter == 'top':
            new_col_order = []
            len_df = data.count()
            if p:
                for idx,col in enumerate(data.columns):
                    if count_rows_values(data)[idx] / len_df >= p:
                        new_col_order.append(col)

            if n:
                raise NotImplementedError

        elif _filter == 'bottom':
            new_col_order = []
            len_df = data.count()
            if p:
                for idx,col in enumerate(data.columns):
                    if count_rows_values(data)[idx] / len_df <= p:
                        new_col_order.append(col)

            if n:
                raise NotImplementedError

        return data.select(*new_col_order)  

    @staticmethod
    def count_rows_values(data:pyspark.sql.DataFrame):
        '''Does pandas equivalent of .count(axis='rows') but for pyspark'''
        counts = data.select(*[F.count(col).alias(col) for col in data.columns]).toPandas().values[0]

        return pd.Series(index=data.columns, data=counts)

    def plot_heatmap_missing(self, cols:list=None):
        """Plots a heatmap of correlation of missing values
        
        Args:
            cols: list of columns to plot missing correlation, if not specified, uses full dataset 
        
        """
        if not cols:
            cols = self.data.columns

        data = self.data.select(*cols)
        data = self.null_filter(data)
        data = self.null_sort(data, sort=None)

        col_counts = self.count_rows_values(data)
        if isinstance(data, pyspark.sql.DataFrame):
            data = data.toPandas()



        data_small = data[col_counts.index]
        corr_mat = data_small.isnull().corr()
        if corr_mat.sum().sum() == 0:
            logging.info('No missing values or empty dataframe')
            return 
        mask = np.zeros_like(corr_mat)
        mask[np.triu_indices_from(mask)] = True

        fig = go.Figure(go.Heatmap(
            z=corr_mat.values,
            x=corr_mat.columns,
            y=corr_mat.columns,
            zmin=-1,
            zmax=1, 
            colorscale=colorcet.coolwarm
        ), layout=go.Layout(
            autosize=False,
            width=800,
            height=800,
            title='Missing Value Correlations'
        ))

        fig.show()

In [499]:
dd.show()

+---+----+-----+------------------+--------------------+----+----+----------+------------+
| id|year|month|              num1|                num2|cat1|cat2|bin_target|multi_target|
+---+----+-----+------------------+--------------------+----+----+----------+------------+
|  A|2023|  1.0|1002.3989695645216|   78.28981021365396|   c|   A|         0|          -1|
|  A|2023|  2.0| 945.2920324240617|  -48.19393418442653|   a|   A|         1|           1|
|  A|2023|  3.0| 803.0476168393415|  -31.00385061597097|   a|   A|         0|           1|
|  A|2023|  4.0| 837.8984800348729| -319.63065707325995|   a|   A|         1|           1|
|  A|2023|  5.0| 842.3216571543076| -16.540609711770397|   c|   C|         1|          -1|
|  A|2023|  6.0| 843.5387529856456|  202.61459610296797|   b|   A|         1|           1|
|  A|2023|  7.0| 886.5776484723364|-0.09885695743611222|   c|   B|         1|           0|
|  A|2023|  8.0|1052.1454089179265|  -75.38275449654631|   c|   B|         1|           1|

In [500]:
s = SparkExplorer(
    data=dd,
    target='bin_target',
    target_type='binary'
)

In [501]:
s.plot_hist(['num1','num2'], n_bins=5)

In [502]:
help(s.plot_box)

Help on method plot_box in module __main__:

plot_box(cols: Union[List, str] = None) method of __main__.SparkExplorer instance
    Plots quartiles using the make_box function
    
    Args:
        cols: list of columns to plot for, if not specified defaults to all



In [503]:
s.plot_box(['num1', 'num2'])

In [504]:
s.plot_corr_map(cols=['num1', 'num2'])

In [505]:
s.plot_bar(['cat1'])

In [506]:
s.plot_heatmap_missing()

In [507]:
s.plot_corr_map()

/tmp/ipykernel_1167145/2175556738.py:404: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [508]:
import json
print(json.dumps(s.make_box(['num1', 'num2']), indent=4))

{
    "num1": {
        "0": {
            "q00": 803.0476168393415,
            "q01": 803.0476168393415,
            "q25": 907.2920695654886,
            "q50": 959.0112145260407,
            "q75": 1059.4072543931134,
            "q99": 1197.3055442349162,
            "q100": 1197.3055442349162
        },
        "1": {
            "q00": 769.4188139185309,
            "q01": 769.4188139185309,
            "q25": 932.2965098911496,
            "q50": 1003.9672686831474,
            "q75": 1048.7928364881604,
            "q99": 1308.6366589738643,
            "q100": 1308.6366589738643
        }
    },
    "num2": {
        "0": {
            "q00": -232.3261679023198,
            "q01": -232.3261679023198,
            "q25": -60.01249303350001,
            "q50": -22.540583256507958,
            "q75": 63.15884542190935,
            "q99": 155.3489810717883,
            "q100": 155.3489810717883
        },
        "1": {
            "q00": -319.63065707325995,
            "q01": -3

In [509]:
dd.show()

+---+----+-----+------------------+--------------------+----+----+----------+------------+
| id|year|month|              num1|                num2|cat1|cat2|bin_target|multi_target|
+---+----+-----+------------------+--------------------+----+----+----------+------------+
|  A|2023|  1.0|1002.3989695645216|   78.28981021365396|   c|   A|         0|          -1|
|  A|2023|  2.0| 945.2920324240617|  -48.19393418442653|   a|   A|         1|           1|
|  A|2023|  3.0| 803.0476168393415|  -31.00385061597097|   a|   A|         0|           1|
|  A|2023|  4.0| 837.8984800348729| -319.63065707325995|   a|   A|         1|           1|
|  A|2023|  5.0| 842.3216571543076| -16.540609711770397|   c|   C|         1|          -1|
|  A|2023|  6.0| 843.5387529856456|  202.61459610296797|   b|   A|         1|           1|
|  A|2023|  7.0| 886.5776484723364|-0.09885695743611222|   c|   B|         1|           0|
|  A|2023|  8.0|1052.1454089179265|  -75.38275449654631|   c|   B|         1|           1|

In [510]:
dd.select(F.percentile_approx('num1', [0.00, 0.01, 0.25, 0.50, 0.75, 0.99, 1.00]).alias('percentiles')).collect()[0].percentiles

[769.4188139185309,
 769.4188139185309,
 927.8292347661644,
 997.9474040470974,
 1052.1454089179265,
 1244.8423950642211,
 1308.6366589738643]

## Pandas

In [539]:
import pandas 

class PandasExplorer(Explorer):
    def __init__(self, 
        data: pandas.DataFrame,
        target: str,
        target_type: str,
        date_cols:Union[List, str] = None,
        date_format: str = None, 
    ):
        '''Pandas-specific implementation of Explorer


        Args:
            data: pandas dataframe
            target: name of target column
            target_type: one of "continuous", "categorical" or "binary"
            date_cols: either a list of ['year', 'month'] OR a single column
            date_format: one of "american", "julian" or "iso" with the dash/dot variations
        
        Raises:
            ValueError: if target-type is not valid

        '''

        self._validate_target_type(target_type)
        self.target_vals = None

        if date_cols and date_format:
            data = PandasDateUtil.create_yearmonth(data, date_cols=date_cols, format=date_format)

        if target_type in ('binary', 'categorical'):
            self.target_vals = self._get_target_vals(data, target)

        self.data = data
        self.target = target
        self.target_type = target_type

    def _get_target_vals(self, data, target):
        return list(data[target].unique())

    def make_bar(self, cols: Union[List, str]):
        if isinstance(cols, str):
            cols = [cols]

        data = self.data.assign(count=1)
        if self.target_type in ('binary', 'categorical'):
            return data.groupby([self.target, *cols], as_index=False).count()
        else:
            return data.groupby(cols, as_index=False).count()
        

    def plot_bar(self, cols: Union[List, str]):
        grouped =self.make_bar(cols)
        print(grouped)
        fig = go.Figure()
        if self.target_type in ('continuous'):
            fig.add_trace(go.Bar(
                x=cols,
                y=grouped.loc[:, 'count']
            ))
        else:
            for target_val in self.target_vals:
                fig.add_trace(go.Bar(
                    x=grouped[col].unique(),
                    y=grouped.loc[grouped[self.target]==target_val, 'count'],
                    name=f'Target: {target_val}'
                ))
        fig.show()

    def make_box(self, cols: Union[List, str]) -> Dict:
        if self.target_type in ('binary', 'categorical'):
            return self._make_box_categorical_target(cols)
        else:
            return self._make_box_continuous_target(cols)

    def _make_box_categorical_target(self, cols):
        data = self.data.groupby(self.target)[cols].quantile((0.00, 0.01, 0.25, 0.50, 0.75, 0.99, 1.00)).reset_index()
        quantile_dict = {}
        for col in cols:
            target_dict = {}
            for target in data[self.target].unique():
                target_q = data.loc[data[self.target] == target, :]
                q00, q01, q25, q50, q75, q99, q100 = target_q[col].values
                target_dict[target] = {
                    'q00': q00,
                    'q01': q01,
                    'q25': q25,
                    'q50': q50,
                    'q75': q75,
                    'q99': q99,
                    'q100': q100
                }

            quantile_dict[col] = target_dict

        return quantile_dict

    def _make_box_continuous_target(self, cols):
        data = self.data[cols].quantile((0.00, 0.01, 0.25, 0.50, 0.75, 0.99, 1.00)).reset_index()
        quantile_dict = {}
        for col in cols:
            q00, q01, q25, q50, q75, q99, q100 = data[col].values
            quantile_dict[col] = {
                'q00': q00,
                'q01': q01,
                'q25': q25,
                'q50': q50,
                'q75': q75,
                'q99': q99,
                'q100': q100
            }


        return quantile_dict


    
    def make_hist(self, cols: Union[List, str], bins=None, n_bins=None) -> Dict:

        if bins is not None and n_bins is not None:
            raise ValueError('specify either n_bins (integer) or bins (bin edges), not both')

        if not n_bins and not bins:
            n_bins = 10

        if self.target_type in ('categorical', 'binary'):
            return self._make_hist_categorical_target(cols, bins, n_bins)
        else:
            return self._make_hist_continuous_target(cols, bins, n_bins)

    def _make_hist_continuous_target(self, cols:Union[list, str], bins=None, n_bins=None) -> Dict:
        if not isinstance(cols, list):
            cols = [cols]

        col_dict = {}
        for col in cols: 
            col_dict[col] = {'bins': None, 'edges': None}

            col_dict['bins'], col_dict['edges'] = np.histogram(self.data.loc[:, col], bins=n_bins if n_bins else bins)

        return col_dict

    def _make_hist_categorical_target(self, cols:Union[List, str], bins=None, n_bins=None) -> Dict:

        if not isinstance(cols, list):
            cols = [cols]
            
        col_dict = {}
        for col in cols:
            target_dict = {}
            for target_val in self.target_vals:
                target_dict[target_val] = {'bins': None, 'edges': None}
                target_dict[target_val]['bins'], target_dict[target_val]['edges']  = np.histogram(self.data.loc[self.data[self.target]==target_val, col], bins=n_bins if n_bins else bins)

            col_dict[col] = target_dict

        return col_dict

    def null_filter(self, data, _filter: str = None, p: float = None, n: float=None) -> pandas.DataFrame:
        if _filter == 'top':
            if p:
                data = data.iloc[:, [c >= p for c in data.count(axis='rows').values / len(data)]]
            if n:
                raise NotImplementedError
        elif _filter == 'bottom':
            if p:
                data= data.iloc[:, [c <= p for c in data.count(axis='rows').values / len(data)]]
            if n:
                raise NotImplementedError

        return data

    def null_sort(self, data, sort=None):
        """
        Sorts a DataFrame according to its nullity, in either ascending or descending order.
        :param df: The DataFrame object being sorted.
        :param sort: The sorting method: either "ascending", "descending", or None (default).
        :return: The nullity-sorted DataFrame.
        """
        if sort is None:
            return data 
        elif sort not in ['ascending', 'descending']:
            raise ValueError('The "sort" parameter must be set to "ascending" or "descending".')

        if sort == 'ascending':
            return data.iloc[np.argsort(data.count(axis='columns').values), :]
        elif sort == 'descending':
            return data.iloc[np.flipud(np.argsort(data.count(axis='columns').values)), :]
        
    def plot_heatmap_missing(self, cols: list = None) -> None:
        if not cols:
            cols = self.data.columns

        data = self.data[cols]
        data = self.null_filter(data)
        data = self.null_sort(data)

        data = data.iloc[:, [i for i, n in enumerate(np.var(data.isnull(), axis='rows')) if n > 0]]

        corr_mat = data.isnull().corr()
        mask = np.zeros_like(corr_mat)
        mask[np.triu_indices_from(mask)] = True

        fig = go.Figure(go.Heatmap(
            z=corr_mat.values,
            x=corr_mat.columns,
            y=corr_mat.columns,
            zmin=-1,
            zmax=1, 
            colorscale=colorcet.coolwarm
        ), layout=go.Layout(
            autosize=False,
            width=800,
            height=800,
            title='Missing Value Correlations'
        ))

        fig.show()

In [540]:
df.head()

,id,year,month,num1,num2,cat1,cat2,bin_target,multi_target
0,A,2023,1.0,1002.398970,78.289810,c,A,0,-1
1,A,2023,2.0,945.292032,-48.193934,a,A,1,1
2,A,2023,3.0,803.047617,-31.003851,a,A,0,1
3,A,2023,4.0,837.898480,-319.630657,a,A,1,1
4,A,2023,5.0,842.321657,-16.540610,c,C,1,-1


In [541]:
p = PandasExplorer(
    data=df, 
    target='cat1', 
    target_type='categorical',
) 

In [542]:
df[['num1', 'num2']].quantile((0.25, 0.50, 0.75))

,num1,num2
0.25,930.838386,-75.402901
0.50,1000.173187,-6.308107
0.75,1053.140438,62.329832


In [543]:
p.plot_box(cols=['num1'])

In [544]:
p.plot_hist(['num1'], n_bins=10)

In [545]:
p.plot_heatmap_missing()